In [1]:
from z3 import *
import numpy as np
import re
import time
import os

In [2]:
# -------READ DATA FROM FILE-----
# Read instance and extract data
lines = []
instance_name = 'pdf' #INSERT INSTANCE NAME HERE
with open(f"../in/{instance_name}.dzn") as f: 
    for line in f:
        line = re.sub("[^0123456789\.\ -]","",line)
        line = line.strip()
        lines.append(line)
# line 1: m
m = int(lines[0])
n = int(lines[1])
l = [int(s) for s in lines[2].split()]
w = [int(s) for s in lines[3].split()]
x = [int(s) for s in lines[4].split()]
y = [int(s) for s in lines[5].split()]

print('m', m)
print('n', n)
print('capacities', l)
print('weights', w)

lines = []

m 3
n 7
capacities [15, 10, 7]
weights [3, 2, 6, 8, 5, 4, 4]


In [3]:
# --------- FUNCTIONS ---------

x = x[-1:] + x[:-1]
y = y[-1:] + y[:-1]
 
def manhattan_distance(x1, y1, x2, y2):
    distance = 0
    absX = abs(x2 - x1)
    absY = abs(y2 - y1)
    distance = absX + absY
    return distance
 
def create_distance_matrix(listX, listY):
    distance_matrix = np.zeros( (len(listX), len(listX)) )
    for i in range(len(listX)):
        for j in range(len(listX)):
            distance_matrix[i, j] = manhattan_distance(listX[i], listY[i], listX[j], listY[j])
 
    return distance_matrix.astype('int').tolist()
 
# --------- VARIABLES ---------
 
distance_matrix = create_distance_matrix(x, y)
print(np.array(distance_matrix))

[[0 2 3 3 4 3 4 4]
 [2 0 3 3 6 5 6 6]
 [3 3 0 4 3 4 7 7]
 [3 3 4 0 7 6 3 5]
 [4 6 3 7 0 3 6 6]
 [3 5 4 6 3 0 3 3]
 [4 6 7 3 6 3 0 2]
 [4 6 7 5 6 3 2 0]]


In [4]:
V = n+2*m
#F_start = n+1
#F_end = n+m
#L_start = n+m+1
#L_end = n+2*m

F_start = n
F_end = n+m-1
L_start = n+m
L_end = n+2*m-1

In [5]:
solver = Solver()
#solver = Optimize()
solver.set("sat.local_search_threads", 16)
c = Int("c")
Z = IntSort()
capacities = IntVector("capacities", m)
weights = IntVector("weights", n)
distance_matrix_sort = [[None for _ in  range(n+1)] for _ in range(n+1)]
for i in range(n+1):
    for j in range(n+1):
        distance_matrix_sort[i][j] = Int(f"distance_matrix_{i}_{j}")
        solver.add(distance_matrix_sort[i][j] == distance_matrix[i][j])
for i in range(m):
    solver.add(capacities[i] == l[i])
for i in range(n):
    solver.add(weights[i] == w[i])
    
# VARIABLES
#array[V] of var V: p; %predecessor array
#array[V] of var V: s; %successor array
#array[V] of var COURIERS: v; %vehicle array
#array[V] of var 0..max(capacities): weights; %quantity array

p = IntVector("p", V)
s = IntVector("s", V)
v = IntVector("v", V)
#bins = IntVector("bins", n)
q = IntVector("q", V)

for i in range(V):
    solver.add(And(p[i] <V, p[i] >= 0 ))
    solver.add(And(s[i] <V, s[i] >= 0 ))
    solver.add(And(v[i] <m, v[i] >= 0 ))
    solver.add(And(q[i] <max(l), q[i] >= 0 ))
    
for k in range(m):
    solver.add(p[n+k] == n+m+k)

solver.add(Distinct(p))
solver.add(Distinct(s))

for i in range(V):
    for x in range(V):
        if((i < F_start or i > F_end) and (x < F_start or x > F_end)):
            solver.add(Implies(p[i] == x, s[x] == i))        #constraint forall (i in V diff F) (s[p[i]]=i);
            solver.add(Implies(p[i] == x, v[i] == v[x]))     #constraint forall (i in V diff F) (v[i] == v[p[i]]);
    for x in range(V):
        if((i < L_start) and (x < L_start)):
            solver.add(Implies(s[i] == x, p[x]==i))          #constraint forall (i in V diff L) (p[s[i]]=i);
            solver.add(Implies(s[i] == x, v[i] == v[x]))     #constraint forall (i in V diff L) (v[i] == v[s[i]]);

# constraint for same vehicle for each courier
for k in range(m):
    solver.add(And(v[n+k] == v[n+m+k], v[n+m+k]==k))

# circuit constraint
order = IntVector("order", V)
solver.add(order[0]==n)
solver.add(Distinct(order))
for i in range(V):
    solver.add(And(order[i] <V, order[i] >= 0 ))
    solver.add(s[i]!=i)
    for j in range(V):
        solver.add(Implies(And(s[i] == j, order[i] != V-1), order[j] == order[i]+1))
    solver.add(Implies(order[i] == V-1, s[i] == n))


# bin packing
for k in range(m):
    res = Int(f"load_{k}")
    solver.add(res == Sum([If(v[i] == k, weights[i], 0) for i in range(n)]))
    solver.add(res <= capacities[k])

total_distance = Int("total_distance")
solver.add(total_distance == Sum([If(And(Or(i<n,i>n+m-1),j == p[i]), distance_matrix_sort[0 if i>=n else i+1][0 if j>=n else j+1], 0) for i in range(V) for j in range(V)]))

solver.add(total_distance >= int(2*np.matrix(distance_matrix).max()))
            
#solver.add(Sum(res) == total_distance)
print("Model created.")
#solver.minimize(total_distance)
#print(solver.check())
#model = solver.model()

Model created.


In [6]:
def get_tour_rec(p, last_i, curr_i):
    if last_i == curr_i:
        return []
    else:
        el = curr_i
        if curr_i > n:
            el = -1
        return [el+1] + get_tour_rec(p, last_i, p[curr_i])

def get_tour(p, last_i):
    return get_tour_rec(p, last_i, p[last_i]) + [0]

In [7]:
l = 0
r = 1000000
mid = 0
while l<=r:
    print(l,r, int(np.ceil(mid)))
    mid = l + (r-l)/2
    if str(solver.check())=='sat':
        model = solver.model()
        r = int(str(model.evaluate(total_distance)))
        mid = l + (r-l)/2
        
        # output formatting
        res_tour = []
        p_tour = []
        for i in range(V):
            visit = int(str(model.evaluate(p[i])))
            p_tour.append(visit)
        for k in range(m):
            res_tour.append(get_tour(p_tour, n+k))
        print('tot_d', r, end='\n\n')
        #write best solution so far to output txt file
        with open("../out/" + instance_name + '_out.txt', 'a') as f:
            f.write(str(r) + ' ')
            f.write(str(res_tour) + '\n\n')

        # resume search
        solver.push()
        solver.add(total_distance < int(np.ceil(mid)))
    else:
        solver.pop()
        solver.push()
        l = mid+1
        solver.add(total_distance < int(np.ceil(mid)))



0 1000000 0
tot_d 38

0 38 19
20.0 38 19
30.0 38 29
35.0 38 34
37.5 38 37
tot_d 34



In [8]:
p_tour = []
s_tour = []
for i in range(V):
    visit = int(str(model.evaluate(p[i])))
    p_tour.append(visit)
    visit = int(str(model.evaluate(s[i])))
    s_tour.append(visit)
print('p index =', [i for i in range(0, len(p_tour))])
print('p_array =', p_tour)
print('s_array =', s_tour)
res_tour = []
for k in range(m):
    res_tour.append(get_tour(p_tour, n+k))

print(res_tour)
res_total_distance = model.evaluate(total_distance)
print(res_total_distance)


#print(solver.statistics()) 

p index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
p_array = [6, 8, 7, 1, 5, 2, 9, 10, 11, 12, 4, 3, 0]
s_array = [12, 3, 5, 11, 10, 4, 0, 2, 1, 6, 9, 7, 8]
[[0, 5, 6, 3, 0], [0, 4, 2, 0], [0, 1, 7, 0]]
34


In [9]:
### to solver independent language

def toSMT2(f, status="unknown", name="smt2", logic=""):
  v = (Ast * 0)()
  if isinstance(f, Solver):
    a = f.assertions()
    if len(a) == 0:
      f = BoolVal(True)
    else:
      f = And(*a)
  return Z3_benchmark_to_smtlib_string(f.ctx_ref(), name, logic, status, "", 0, v, f.as_ast())

if False:
  solv_indep_code = toSMT2(solver, name=instance_name)
  smt_out_path = f'./smtlib_code/{instance_name}.smt2'
  with open(smt_out_path, 'a') as smt_out:
      smt_out.write(solv_indep_code)
      print(f'smtlib2 code written to file {smt_out_path}')